# **Stable Diffusion** 🎨

Stable Diffusion은 CompVis, Stability AI, 그리고 LAION에서 연구원과 엔지니어들에 의해 만들어진 텍스트-이미지 잠재 확산 모델입니다. 이 모델은 LAION-5B 데이터베이스의 일부에서 가져온 512x512 이미지들에 대해 훈련되었습니다. 모델은 텍스트 프롬프트에 따라 모델을 조건화하기 위해 고정된 CLIP ViT-L/14 텍스트 인코더를 사용합니다. 860M UNet과 123M 텍스트 인코더를 가진 이 모델은 상대적으로 가벼우며 적어도 10GB VRAM을 가진 GPU에서 실행됩니다.
더 많은 정보는 모델 카드를 참조하세요.

## 1. `StableDiffusionPipeline` 사용하는 법

Stable Diffusion이 어떻게 작동하는지에 대한 이론적인 측면에 뛰어들기 전에 이 섹션에서는 단 몇 줄의 코드만으로 텍스트에서 이미지로의 추론을 실행하는 방법을 보여줍니다!

### Setup

먼저, 이 노트북을 실행하는 데 GPU 런타임을 사용하고 있는지 확인해 주세요, 그래야 추론이 훨씬 더 빨라집니다. 아래 명령이 실패한다면, 위의 런타임 메뉴를 사용하고 런타임 유형 변경을 선택하세요.

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.27.2 #24.03.21 기준 최신버전
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install fifty accelerate

### Stable Diffusion Pipeline

StableDiffusionPipeline은 몇 줄의 코드만으로 텍스트에서 이미지를 생성할 수 있는 엔드-투-엔드 추론 파이프라인입니다.

먼저, 모델의 모든 구성 요소의 사전 훈련된 가중치를 로드합니다.


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id)

다음으로, 추론을 더 빠르게 하기 위해 파이프라인을 GPU로 옮겨봅시다.








In [ ]:
pipe = pipe.to("cuda")

모든 호출에서 대략적으로 동일한 이미지를 재현할 수 있도록, 생성기(generator)를 사용해봅시다.

이미지를 생성하기 위해서는 __call__ 메소드를 사용해야 합니다. 여기에 어떤 인자를 전달할 수 있는지 더 잘 이해하기 위해 해당 [문서](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.__call__)을 살펴볼 수 있습니다.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)

prompt = "portrait photo of a old warrior chief"

image = pipe(prompt, generator=generator).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can do either save it such as:
image.save(f"warrior chief.png")

# or if you're in a google colab you can directly display it with
image

이 작업은 T4 GPU에서 대략 30초 정도 걸렸습니다(할당된 GPU가 T4보다 좋다면 추론 속도가 더 빨라질 수 있습니다).

위에서 실행한 기본 설정은 전체 float32 정밀도를 사용하고 기본 50 단계 추론을 실행했습니다. 가장 쉬운 속도 향상 방법은 float16(또는 half) 정밀도로 전환하고 단순히 더 적은 추론 단계를 실행하는 것입니다. 이제 float16에서 모델을 로드해봅시다.

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

다시 실행해봅니다.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)

image = pipe(prompt, generator=generator).images[0]
image

이 방법은 거의 동일한 이미지 품질을 유지하면서 거의 세 배 빠른 속도를 제공합니다. 파이프라인을 float16으로 실행할 것이 권장됩니다. 이로 인한 품질 저하는 매우 적습니다.

다음으로, 정말로 50개의 추론 단계를 사용해야 하는지, 아니면 훨씬 적은 단계를 사용할 수 있는지 살펴봅시다. Stable Diffusion 파이프라인과 호환되는 모든 스케줄러를 살펴보겠습니다.

In [ ]:
pipe.scheduler.compatibles

Stable Diffusion 모델은 기본적으로 PNDMScheduler를 사용하는데, 이는 보통 약 50개의 추론 단계가 필요합니다. 하지만 DPMSolverMultistepScheduler와 같은 더 성능이 좋은 스케줄러는 오직 약 20 또는 25개의 추론 단계만 필요합니다.

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

추론 스텝을 20으로 낮춰 실행해보겠습니다.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)

image = pipe(prompt, generator=generator, num_inference_steps=20).images[0]
image

프롬프트에 디테일을 추가해보겠습니다.

In [ ]:
prompt = "portrait photo of a old warrior chief"
prompt += ", tribal panther make up, blue on red, side profile, looking away, serious eyes"

In [ ]:
prompt

그리고 일반적으로 더 높은 품질의 이미지를 생성하는 데 도움이 되는 몇 가지 토큰도 추가해봅시다.








In [ ]:
prompt += " 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta"
prompt

실행해봅시다.

In [ ]:
image = pipe(prompt, generator=generator, num_inference_steps=20).images[0]
image

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_cols = 3
num_rows = 4

prompt = [prompt] * num_cols
all_images = []
for i in range(num_rows):
  images = pipe(prompt, generator=generator, num_inference_steps=20).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

다른 프롬프트도 시도해봅니다.

In [ ]:
prompt = "a photo of an astronaut riding a horse on mars"
image =pipe(prompt, generator=generator, num_inference_steps=20).images[0]

image.save("astronaut_rides_horse.png")

image

In [ ]:
prompt = 'some knights riding horses in an beautiful ancient forest.'
image = pipe(prompt, generator=generator, num_inference_steps=20).images[0]

# Now to display an image you can do either save it such as:
image.save(f"knights.png")

# or if you're in a google colab you can directly display it with
image

파이프라인 호출의 또 다른 매개변수는 guidance_scale입니다. 이는 프롬프트의 강제력을 증가시키고 전반적인 샘플 품질을 향상시키는 방법입니다. 간단히 말해, 생성을 프롬프트와 더 잘 일치하게 만듭니다. 7이나 8.5와 같은 숫자는 좋은 결과를 줍니다. 만약 매우 큰 숫자를 사용하면 이미지가 좋아 보일 수는 있지만, 다양성이 줄어듭니다.

### 직사각형 이미지를 생성하기

Stable Diffusion은 기본적으로 512 × 512 픽셀의 이미지를 생성합니다. 하지만 height와 width 인수를 사용하여 기본값을 쉽게 재정의할 수 있으므로, 세로나 가로 비율의 직사각형 이미지를 만들 수 있습니다.

좋은 이미지 크기를 선택하기 위한 몇 가지 권장 사항은 다음과 같습니다:

- 높이와 너비가 모두 8의 배수인지 확인하세요.
- 512보다 낮은 값으로 가면 이미지 품질이 떨어질 수 있습니다.

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

image = pipe(prompt, height=512, width=768).images[0]
image

## 2. Stable Diffusion의 이론적 파트

Stable Diffusion은 Latent Diffusion이라고 불리는 특정 유형의 확산 모델에 기반을 두고 있습니다.




Latent diffusion model의 주요 구성은 아래와 같습니다.

1. An autoencoder (VAE).
2. A [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq).
3. A text-encoder, *e.g.* [CLIP's Text Encoder](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel).

**1. The autoencoder (VAE)**

VAE 모델에는 인코더와 디코더의 두 부분이 있습니다. 인코더는 이미지를 낮은 차원의 잠재 표현으로 변환하는 데 사용되며, 이는 U-Net 모델의 입력으로 사용됩니다. 반대로, 디코더는 잠재 표현을 다시 이미지로 변환합니다.

**2. The U-Net**

U-Net에는 ResNet 블록으로 구성된 인코더 부분과 디코더 부분이 있습니다. 인코더는 이미지 표현을 더 낮은 해상도의 이미지 표현으로 압축하고 디코더는 낮은 해상도의 이미지 표현을 원래의 더 높은 해상도의 이미지 표현으로 다시 디코딩하여 이론적으로 덜 노이즈가 있는 이미지를 복원합니다. 구체적으로, U-Net 출력은 노이즈를 예측하는데, 이를 사용하여 예측된 더 깨끗한 이미지 표현을 계산할 수 있습니다.

U-Net이 다운샘플링하는 동안 중요한 정보를 잃지 않도록, 보통 인코더의 다운샘플링 ResNets에서 디코더의 업샘플링 ResNets로 짧은 연결(short-cut connections)이 추가됩니다. 또한, 안정된 확산 U-Net은 텍스트 임베딩에 따라 출력을 조건화할 수 있는데, 이는 ResNet 블록 사이에 보통 추가되는 cross-attention 층을 통해 가능합니다. cross-attention 층은 U-Net의 인코더 및 디코더 부분 양쪽에 추가됩니다.

**3. The Text-encoder**

텍스트 인코더는 예를 들어 "우주인이 말을 타는 모습"과 같은 입력 프롬프트를 U-Net이 이해할 수 있는 임베딩 공간으로 변환하는 역할을 합니다. 보통 간단한 트랜스포머 기반 인코더를 사용하여 입력 토큰의 시퀀스를 잠재적인 텍스트 임베딩의 시퀀스로 매핑합니다.

왜 잠재 확산(Latent Diffusion)은 빠르고 효율적인가?

잠재 확산 모델의 U-Net이 저차원 공간에서 작동하기 때문에 픽셀 공간 확산 모델에 비해 메모리와 계산 요구량이 크게 감소합니다. 예를 들어 Stable Diffusion에서 사용되는 오토인코더는 축소 계수가 8입니다. 즉, (3, 512, 512) 크기의 이미지가 잠재 공간(latent space)에서는 (3, 64, 64)가 되므로 8×8=64배 적은 메모리를 필요로 합니다.

이것이 바로 16GB Colab GPU에서도 512 × 512 이미지를 신속하게 생성할 수 있는 이유입니다!

**Stable Diffusion during inference**


<p align="left">
<img src="https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/stable_diffusion.png" alt="sd-pipeline" width="500"/>
</p>

Stable Diffusion 모델은 잠재 시드(latent seed)와 텍스트 프롬프트를 모두 입력으로 받습니다. 그런 다음 잠재 시드는 64×64 크기의 랜덤 잠재 이미지 표현을 생성하는 데 사용되며, 텍스트 프롬프트는 CLIP의 텍스트 인코더를 통해 77×768 크기의 텍스트 임베딩으로 변환됩니다.

다음으로 U-Net은 텍스트 임베딩을 조건으로 하면서 랜덤 잠재 이미지 표현을 반복적으로 노이즈 제거합니다. U-Net의 출력인 노이즈 잔차는 스케줄러 알고리즘을 통해 노이즈가 제거된 잠재 이미지 표현을 계산하는 데 사용됩니다.

노이즈 제거 과정은 약 50번 반복되어 점진적으로 잠재 이미지의 표현 성능을 개선합니다. 완료되면 잠재 이미지 표현은 변이 오토인코더(variational autoencoder)의 디코더 부분에 의해 디코딩됩니다.

## 3. Stable diffusion 내부 실행 과정 보기


In [ ]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

전 훈련된 모델에는 완전한 확산 파이프라인을 설정하는 데 필요한 모든 구성 요소가 포함되어 있습니다. 이 구성 요소들은 다음 폴더에 저장됩니다:

- text_encoder: 안정적 확산은 CLIP를 사용하지만, 다른 확산 모델은 BERT와 같은 다른 인코더를 사용할 수 있습니다.
- tokenizer: 텍스트 인코더 모델에서 사용하는 것과 일치해야 합니다.
- scheduler: 훈련 중에 이미지에 점진적으로 노이즈를 추가하는 데 사용되는 스케줄링 알고리즘입니다.
- unet: 입력의 잠재적 표현을 생성하는 데 사용되는 모델입니다.
- vae: 잠재적 표현을 실제 이미지로 디코딩하는 데 사용할 오토인코더 모듈입니다.
- from_pretrained를 사용할 때 subfolder 인자를 지정함으로써 저장된 폴더를 참조하여 이 구성 요소들을 불러올 수 있습니다.

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

모델들을 GPU로 이동시킵니다.

In [ ]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

이제 이미지를 생성하는 데 사용할 매개변수를 정의합니다.
guidance_scale은 Imagen 논문의 수식 (2)에서 안내 가중치(guidance weight) w와 유사하게 정의됩니다. guidance_scale == 1은 classifier-free guidance 를 수행하지 않는 것과 일치합니다. 여기서는 이전과 같이 7.5로 설정합니다.
이전 예제와 달리, num_inference_steps를 100으로 설정하여 좀 더 선명한 이미지를 얻습니다.

In [ ]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512                        # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion

num_inference_steps = 100            # Number of denoising steps

guidance_scale = 7.5                # Scale for classifier-free guidance

generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise

batch_size = 1

우선, 프롬프트에 대한 텍스트 임베딩을 얻습니다. 이 임베딩은 UNet 모델을 조건화(conditioning)하는 데 사용됩니다.

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

In [ ]:
text_embeddings.shape

또한, classifier-free guidance를 위한 unconditional 텍스트 임베딩도 가져올 것입니다. 이것은 null text(빈 텍스트)에 대한 임베딩일 뿐입니다. 이들은 conditional 텍스트 임베딩(배치 크기와 시퀀스 길이)과 동일한 모양을 가져야 합니다.

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

In [ ]:
uncond_embeddings.shape

두 번의 실행을 피하기 위해 둘을 하나의 배치로 연결할 수 있습니다.








In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

In [ ]:
text_embeddings.shape

초기 noise latent map을 생성합니다.

In [ ]:
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)

In [ ]:
latents.shape

64×64 크기의 latent map이 예상되며, 모델은 나중에 이 잠재적 표현을 512×512 크기의 이미지로 변환할 것입니다.

다음으로, 선택한 num_inference_steps로 스케줄러를 초기화합니다. 이 과정에서는 디노이징 과정 동안 사용될 시그마 값과 정확한 time-step 값을 계산합니다.

In [ ]:
scheduler.set_timesteps(num_inference_steps)

K-LMS 스케줄러는 잠재 변수를 초기 노이즈 시그마 값으로 곱해야 합니다.

In [ ]:
latents = latents * scheduler.init_noise_sigma

In [ ]:
latents.shape

디노이징 루프를 작성합니다.

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input = torch.cat([latents] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents = scheduler.step(noise_pred, t, latents).prev_sample

In [ ]:
latents.shape

이제 vae를 사용하여 생성된 잠재 변수를 이미지로 다시 디코드합니다.

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
  image = vae.decode(latents).sample

이미지를 시각화합니다.

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

In [ ]:
images.shape